# 파파고 번역기 사용하기

- 파파고 번역을 사용하기 위해서는 https://developers.naver.com/main/ 에서 application 등록 선행
- Client ID 와 Client Secret 을 부여 받아 사용
- 한 api 당 글자 수 제한(최대 10,000자)가 있기 때문에 우리가 번역할 데이터의 글자 수가 몇 개인지 먼저 판단
- application 등록은 최대 10개까지 가능

### 1. 데이터 전처리
- 사용 편의를 위하여 해당 파일은 아주 기본적인 전처리 후 올려드리는 파일입니다. (ex. '_comma_', 번역해야 하는 부분 nan값, 불필요한 컬럼 제거 등)
- 하지만, 구체적인 전처리, 올바르지 않은 문장, 겹치는 문장 등은 아직 전처리가 되어있지 않기 때문에 번역 시 해당 부분 참고 부탁 드립니다.

### 2. 번역 가능한 크기로 파일 나눠주기

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
data_path = '../dataset/'

In [6]:
train_df = pd.read_csv(os.path.join(data_path, 'train01.csv'), encoding='utf-8') # test01은 기본적인 전처리는 되어있음
train_df.head(3)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,"Was this a friend you were in love with, or ju...",5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN


In [4]:
def split_data(data):
    '''
    data를 지정한 sep 크기만큼 분리하여 반환하는 함수 
    '''
    sep = 50
    divided_list = [] 
    for idx in range(0, len(data), sep):
        divided_list.append(data[idx:idx + sep])
        
    return divided_list

In [5]:
data_list = split_data(train_df)
data_list[0].head(3)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,"Was this a friend you were in love with, or ju...",5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN


### 3. 파파고 번역기로 데이터 번역하기(영어 → 한글)
- 위해서 슬라이싱 해준 데이터 이용
- 미리 할당받은 어플리케이션의 client_id와 cilent_secret값 이용
- application url 이용

In [6]:
import os
import sys
import urllib.request
import json
from pprint import pprint
from tqdm import tqdm
import requests
import time # 수행시간 측정

#### 3-1. 할당받은 client_id / client_secret 저장
- 앞으로 계속 해당 형태로 이용하게 된다면 리스트를 묶어서 일괄 처리 시켜줄 예정
- 10개의 application은 리스트 형태로, 각각의 application의 id와 secret값은 튜플로 관리

In [7]:
client = dict([])

# 번역기 Client 생성   
# client[key] = [value], value: Client ID, Client Secret
client['app_01'] = ['5FYKQUZOTO189JlPzHQD', 'F0Ldyrtb6M']   
client['app_02'] = ['g23Xj2JPuN69GVGJd1DH', 'HB87ywFSsT']
client['app_03'] = ['b03zg65GS36Wi22KM207', 'k8O5gkUN71']
client['app_04'] = ['NHPVyipVmwbqJ69UIzxb', 'swVGpSU8Ip']
client['app_05'] = ['UkP5AKqLrbrbZejq5m9t', '3teo5JMiEP']
client['app_06'] = ['aTwCHEDi3ybnlTfSfqbk', 'TVjp8xCPWk']
client['app_07'] = ['mchXcBoyU8TBEUDPaMlB', 'b2bNyb0Cb8']
client['app_08'] = ['89E85WEvyXth2_ksHXLr', 'C3u4wCJvcL']
client['app_09'] = ['cZ0nA6WX6A_DNFmDPreb', 'b85n4OYFJr']
client['app_10'] = ['P4em7Un8XQ3V87k5_CQS', '8T8jiACfBV']

In [8]:
client

{'app_01': ['client id', 'client secret']}

#### 3-2. 번역
- 영어가 들어가 있는 열인 'context', 'prompt', 'utterance' 만 번역 → 각각 개별 수행
- 번역 후에는 각각 리스트 형태로 저장

In [9]:
len(data_list), len(data_list[0]), len(data_list[-1])

(1534, 50, 23)

##### * 하나의 어플리케이션이 60개 데이터에 대해 2번의 번역 (prompt, utterance)을 수행하면 9,177번의 API 호출 발생
-> 한 사람 당 하루에 600개 데이터 정도 번역할 수 있음

In [11]:
def papago_translate(cli, data, column):
    result = []
    for i in data[column]:
        encText = urllib.parse.quote(i)
        data = "source=en&target=ko&text=" + encText
        url = "https://openapi.naver.com/v1/papago/n2mt"
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", cli[0])   # client id
        request.add_header("X-Naver-Client-Secret", cli[1])   # client secret
        response = urllib.request.urlopen(request, data=data.encode("utf-8"))
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            res = json.loads(response_body.decode('utf-8')) 
            result.append(res['message']['result']['translatedText'])
        else:
            print("Error Code:" + rescode)
    return result

In [12]:
s_idx = 350   # 매일 실행할 때, 해당 값을 변경해주어야 함 
e_idx = s_idx

prompt_total = [] 
utterance_total = [] 
error_block = []   # 데이터 블록이 10,000글자를 넘는 경우 저장 

for cli in client:
    try:
        prompt_list = papago_translate(client[cli], data_list[e_idx], 'prompt')
        utterance_list = papago_translate(client[cli], data_list[e_idx], 'utterance') 
    except:
        print(f'{cli}의 일일 API 호출량이 초과되었습니다.')
        error_block.append(e_idx)
        e_idx += 1 
        continue
        
    prompt_total.extend(prompt_list)
    utterance_total.extend(utterance_list)   # utterance_total += utterance_list 
    e_idx += 1   # 다음 애플리케이션은 다음 데이터 블록 번역하도록 설정

app_01의 일일 API 호출량이 초과되었습니다.


In [13]:
len(prompt_total), len(utterance_total)

(450, 450)

In [14]:
error_block

[350]

In [15]:
error_df = pd.read_csv(os.path.join(data_path, 'error_block.csv'))

In [16]:
df_idx = len(error_df)

for block in error_block: 
    error_df.loc[df_idx] = block 
    df_idx += 1 

error_df

,error_block
0,30
1,32
2,47
3,59
4,63
5,64
6,68
7,93
8,95
9,96


In [17]:
# error_df = pd.DataFrame(error_block, columns=['error_block'])
error_df.to_csv(os.path.join(data_path, 'error_block.csv'), index=None)

#### 3-3. 번역 결과 새로운 데이터 프레임에 저장

In [18]:
data_list[0][:3]

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,"Was this a friend you were in love with, or ju...",5|5|5_2|2|5,NaN
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN


In [19]:
prompt_total[:3]

['난 내 자신이 너무 자랑스러워.  나는 오늘 새로운 일을 시작했어!',
 '난 내 자신이 너무 자랑스러워.  나는 오늘 새로운 일을 시작했어!',
 '나는 내가 원하는 대학에 들어갈 기회가 있을 때 신이 났다. 난 그저 잘 될 거라고 믿었어요']

#### csv 파일 만들기 

In [20]:
s_idx, e_idx

(350, 360)

In [21]:
new_df = train_df.copy()
new_df = new_df[s_idx * 50: e_idx * 50]
new_df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
17500,hit:2811_conv:5622,2,faithful,When I had to pass my classes to graduate,44,congratulations!,4|5|5_5|5|5,<UNIGRAM>
17501,hit:2811_conv:5622,3,faithful,When I had to pass my classes to graduate,123,Thank you! I didn't know if I was going to be ...,4|5|5_5|5|5,NaN
17502,hit:2811_conv:5622,4,faithful,When I had to pass my classes to graduate,44,i am really happy for you,4|5|5_5|5|5,NaN
17503,hit:2812_conv:5625,1,impressed,A friend of mine wrote a book. I admire the de...,422,A friend just published her own book. I don't ...,5|5|5_5|5|5,NaN
17504,hit:2812_conv:5625,2,impressed,A friend of mine wrote a book. I admire the de...,64,That's awesome! What is the name of the book?,5|5|5_5|5|5,NaN
...,...,...,...,...,...,...,...,...
17995,hit:2915_conv:5830,2,joyful,I'm so happy that my cousin got a full ride sc...,243,That is great! You must be so proud!,5|5|5_5|5|5,NaN
17996,hit:2915_conv:5830,3,joyful,I'm so happy that my cousin got a full ride sc...,269,"Her parents are pretty proud, but I'm more joy...",5|5|5_5|5|5,NaN
17997,hit:2915_conv:5830,4,joyful,I'm so happy that my cousin got a full ride sc...,243,That is nice that you are so supportive of her.,5|5|5_5|5|5,NaN
17998,hit:2915_conv:5831,1,nostalgic,"Every time my best friend comes in town, we go...",243,"Every time my best friend comes back in town, ...",5|5|5_5|5|5,NaN


In [22]:
error_list = []

for idx in error_block:
    error_list.extend(list(range(idx * 50, (idx + 1) *50)))

len(error_list)

50

In [23]:
new_df.drop(error_list, inplace=True)
new_df

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
17550,hit:2834_conv:5668,3,proud,I am so proud of myself. I started a new job ...,160,I know. I had my last job 5 years so it was h...,5|5|5_5|5|5,NaN
17551,hit:2834_conv:5668,4,proud,I am so proud of myself. I started a new job ...,135,"Yeah, people get comfortable even if they don'...",5|5|5_5|5|5,NaN
17552,hit:2835_conv:5671,1,hopeful,I was excited when I had a chance to get into ...,126,4,4|4|5_4|5|4,<UNIGRAM> <NUMERAL>
17553,hit:2835_conv:5671,2,hopeful,I was excited when I had a chance to get into ...,264,I did not understand the number 4,4|4|5_4|5|4,NaN
17554,hit:2835_conv:5671,3,hopeful,I was excited when I had a chance to get into ...,126,"Sorry, I meant I was excited when I got to go ...",4|4|5_4|5|4,NaN
...,...,...,...,...,...,...,...,...
17995,hit:2915_conv:5830,2,joyful,I'm so happy that my cousin got a full ride sc...,243,That is great! You must be so proud!,5|5|5_5|5|5,NaN
17996,hit:2915_conv:5830,3,joyful,I'm so happy that my cousin got a full ride sc...,269,"Her parents are pretty proud, but I'm more joy...",5|5|5_5|5|5,NaN
17997,hit:2915_conv:5830,4,joyful,I'm so happy that my cousin got a full ride sc...,243,That is nice that you are so supportive of her.,5|5|5_5|5|5,NaN
17998,hit:2915_conv:5831,1,nostalgic,"Every time my best friend comes in town, we go...",243,"Every time my best friend comes back in town, ...",5|5|5_5|5|5,NaN


In [24]:
new_df['trans_context'] = new_df['context']
new_df['trans_prompt'] = prompt_total
new_df['trans_utterance'] = utterance_total
new_df = new_df[['conv_id', 'utterance_idx', 'context', 'trans_context', 'prompt', 'trans_prompt', 'speaker_idx', \
                 'utterance', 'trans_utterance', 'selfeval', 'tags']]
new_df

,conv_id,utterance_idx,context,trans_context,prompt,trans_prompt,speaker_idx,utterance,trans_utterance,selfeval,tags
17550,hit:2834_conv:5668,3,proud,proud,I am so proud of myself. I started a new job ...,난 내 자신이 너무 자랑스러워. 나는 오늘 새로운 일을 시작했어!,160,I know. I had my last job 5 years so it was h...,그러게. 난 지난 5년 동안 직장을 다녔기 때문에 그만두고 새로운 것을 시도하기가 ...,5|5|5_5|5|5,NaN
17551,hit:2834_conv:5668,4,proud,proud,I am so proud of myself. I started a new job ...,난 내 자신이 너무 자랑스러워. 나는 오늘 새로운 일을 시작했어!,135,"Yeah, people get comfortable even if they don'...","그래, 사람들은 싫어도 편안해져. 새로운 것을 시도하는 것은 때때로 좋다.",5|5|5_5|5|5,NaN
17552,hit:2835_conv:5671,1,hopeful,hopeful,I was excited when I had a chance to get into ...,나는 내가 원하는 대학에 들어갈 기회가 있을 때 신이 났다. 난 그저 잘 될 거라고...,126,4,4,4|4|5_4|5|4,<UNIGRAM> <NUMERAL>
17553,hit:2835_conv:5671,2,hopeful,hopeful,I was excited when I had a chance to get into ...,나는 내가 원하는 대학에 들어갈 기회가 있을 때 신이 났다. 난 그저 잘 될 거라고...,264,I did not understand the number 4,나는 숫자 4를 이해하지 못했다.,4|4|5_4|5|4,NaN
17554,hit:2835_conv:5671,3,hopeful,hopeful,I was excited when I had a chance to get into ...,나는 내가 원하는 대학에 들어갈 기회가 있을 때 신이 났다. 난 그저 잘 될 거라고...,126,"Sorry, I meant I was excited when I got to go ...","미안, 내 말은 내가 가능성 있는 대학에 가게 되었을 때 흥분했다는 뜻이었어. 내가...",4|4|5_4|5|4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17995,hit:2915_conv:5830,2,joyful,joyful,I'm so happy that my cousin got a full ride sc...,내 사촌이 전액 장학금을 받아서 너무 기뻐! 그녀는 이 기회가 성공하기 위해 정말 ...,243,That is great! You must be so proud!,대단하다! 정말 자랑스러우시겠어요!,5|5|5_5|5|5,NaN
17996,hit:2915_conv:5830,3,joyful,joyful,I'm so happy that my cousin got a full ride sc...,내 사촌이 전액 장학금을 받아서 너무 기뻐! 그녀는 이 기회가 성공하기 위해 정말 ...,269,"Her parents are pretty proud, but I'm more joy...","그녀의 부모님은 꽤 자랑스러워 하지만, 나는 그 어떤 것보다 그녀에게 더 기뻐!",5|5|5_5|5|5,NaN
17997,hit:2915_conv:5830,4,joyful,joyful,I'm so happy that my cousin got a full ride sc...,내 사촌이 전액 장학금을 받아서 너무 기뻐! 그녀는 이 기회가 성공하기 위해 정말 ...,243,That is nice that you are so supportive of her.,네가 그녀를 그렇게 지지하다니 정말 좋다.,5|5|5_5|5|5,NaN
17998,hit:2915_conv:5831,1,nostalgic,nostalgic,"Every time my best friend comes in town, we go...","내 가장 친한 친구가 마을에 올 때마다, 우리는 우리가 만났을 때 함께 일했던 오래...",243,"Every time my best friend comes back in town, ...","나의 가장 친한 친구가 마을에 돌아올 때마다, 우리는 우리가 만났을 때 일하던 오래...",5|5|5_5|5|5,NaN


#### csv 파일 저장 

In [25]:
from datetime import datetime 

now = datetime.now()
file_name = str(now.year) + str(now.month).zfill(2) + str(now.day).zfill(2) + '.csv'
file_name

'20221108.csv'

In [26]:
# 새로운 데이터 셋으로 저장
new_df.to_csv(os.path.join(data_path, file_name),encoding = 'utf-8', index=None)

#### 주 단위 파일 만들기 

In [31]:
columns = ['conv_id', 'utterance_idx', 'context', 'trans_context', 'prompt', 'trans_prompt', 'speaker_idx', \
           'utterance', 'trans_utterance', 'selfeval', 'tags']

tmp = [None] * len(columns)
start_date = 25

In [34]:
week_df = pd.DataFrame([tmp], columns=columns)

for _ in range(7):
    try:
        file_name = str(now.year) + str(now.month).zfill(2) + str(start_date).zfill(2) + '.csv'
        data = pd.read_csv(os.path.join(data_path, file_name))
    except:
        file_name = str(now.year) + (str(now.month - 1).zfill(2)) + str(start_date).zfill(2) + '.csv'
        data = pd.read_csv(os.path.join(data_path, file_name))
    
    start_date += 1 
    week_df = pd.concat([week_df, data])

week_df = week_df[1:]
week_df.reset_index(inplace=True, drop=True)
week_df

,conv_id,utterance_idx,context,trans_context,prompt,trans_prompt,speaker_idx,utterance,trans_utterance,selfeval,tags
0,hit:1934_conv:3868,4,surprised,surprised,I am astonish by Trump.,나는 트럼프에게 놀랐다.,254,"Yes, he's definitely unpredictable and caprici...","네, 그는 확실히 예측할 수 없고 변덕스러워요. 그에 대해 좋아하는 것이 있습니까?",5|4|4_4|5|4,NaN
1,hit:1935_conv:3870,1,anxious,anxious,When I was younger scary movie characters used...,"내가 어렸을 때, 내가 잠들기 전에 무서운 영화 캐릭터들이 내 머릿속에 떠오르곤 했...",119,"My first time watching ""The Sixth Sense"" was w...","내가 ""식스 센스""를 처음 본 것은 10살 때였는데 자기 전에 떨렸거나 불이 없는 ...",5|5|5_5|5|5,NaN
2,hit:1935_conv:3870,2,anxious,anxious,When I was younger scary movie characters used...,"내가 어렸을 때, 내가 잠들기 전에 무서운 영화 캐릭터들이 내 머릿속에 떠오르곤 했...",353,Oh wow that must have kept you awake for a while!,"오, 와, 그것 때문에 한동안 잠을 못 잤나 보구나!",5|5|5_5|5|5,NaN
3,hit:1935_conv:3870,3,anxious,anxious,When I was younger scary movie characters used...,"내가 어렸을 때, 내가 잠들기 전에 무서운 영화 캐릭터들이 내 머릿속에 떠오르곤 했...",119,Yup sweatin bullets haha,"그래, 총알에 땀 흘려 하하.",5|5|5_5|5|5,NaN
4,hit:1935_conv:3870,4,anxious,anxious,When I was younger scary movie characters used...,"내가 어렸을 때, 내가 잠들기 전에 무서운 영화 캐릭터들이 내 머릿속에 떠오르곤 했...",353,"She scared me too, I hope you're not scarred f...","나도 깜짝 놀랐어, 평생 흉터가 없었으면 좋겠어!",5|5|5_5|5|5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3145,hit:2561_conv:5122,2,grateful,grateful,My grandma went out of her way to print out so...,할머니는 나에게 보내려고 일부러 옛날 사진 몇 장을 출력해 주셨다.,395,That's sweet of her! You should put them all t...,정말 친절하시군요! 당신은 그것들을 모두 멋진 사진첩에 넣어야 합니다.,5|5|5_5|5|5,NaN
3146,hit:2561_conv:5122,3,grateful,grateful,My grandma went out of her way to print out so...,할머니는 나에게 보내려고 일부러 옛날 사진 몇 장을 출력해 주셨다.,152,"I am planning on it, but yes she's so kind, I ...","나는 그것을 계획하고 있지만, 응 그녀는 너무 친절해, 나는 운이 좋아.",5|5|5_5|5|5,NaN
3147,hit:2561_conv:5122,4,grateful,grateful,My grandma went out of her way to print out so...,할머니는 나에게 보내려고 일부러 옛날 사진 몇 장을 출력해 주셨다.,395,That is very kind. I'm honestly doubt that my ...,정말 친절하시군요. 나는 솔직히 우리 할머니가 프린터가 무엇인지 알고 있는지 의심스럽다.,5|5|5_5|5|5,NaN
3148,hit:2562_conv:5125,1,annoyed,annoyed,"I ordered a meal from a restaurant, and when I...","식당에서 식사를 주문했는데, 가지러 갔더니 한 번도 못 받았다고 하더군요. 나는 정...",366,I hate when a restaurant claims to never have ...,나는 레스토랑이 당신의 온라인 주문을 받은 적이 없다고 주장하는 것이 싫다. 그리고...,5|5|5_5|5|5,NaN


In [35]:
w_info = str(now.year) + str(now.month).zfill(2) + '_1st.csv'
week_df.to_csv(os.path.join(data_path, w_info),encoding = 'utf-8', index=None)